In [3]:
import pandas as pd 
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import create_extraction_chain
import os
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import openai
from langchain.document_loaders import CSVLoader
from langchain.chains import SequentialChain

In [4]:
os.environ["OPENAI_API_TYPE"]       = ""
os.environ["OPENAI_API_BASE"]       = ""
os.environ["OPENAI_API_KEY"]        = ""
os.environ["OPENAI_API_VERSION"]    = ""

openai.api_type     = os.environ["OPENAI_API_TYPE"]
openai.api_base     = os.environ["OPENAI_API_BASE"]
openai.api_key      = os.environ["OPENAI_API_KEY"] 
openai.api_version  = os.environ["OPENAI_API_VERSION"]

In [5]:
human_template  = "{question}"

In [50]:
quiz_bank = """Article 1: Narendra Modi
Narendra Modi is the current Prime Minister of India, serving since May 2014. He is a member of the Bharatiya Janata Party (BJP) and the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist volunteer organization. Modi's leadership has been marked by economic reforms, including the implementation of the Goods and Services Tax (GST) and the demonetization of high-value currency notes. He has also focused on improving India's infrastructure and promoting digitalization. Modi's policies have been praised for their focus on development, but he has also faced criticism for his handling of religious and social issues.

Article 2: Joe Biden
Joe Biden is the 46th President of the United States, having assumed office in January 2021. He previously served as Vice President under Barack Obama from 2009 to 2017. Biden has been a prominent figure in American politics for decades, serving as a Senator from Delaware from 1973 to 2009. His presidency has been marked by efforts to tackle the COVID-19 pandemic, revitalize the economy, address climate change, and promote racial equity. Biden's administration has also focused on restoring diplomatic relations with allies and rejoining international agreements.

Article 3: Vladimir Putin
Vladimir Putin has been a dominant figure in Russian politics for over two decades. He served as President of Russia from 2000 to 2008, Prime Minister from 2008 to 2012, and President again since 2012. Putin's leadership has been characterized by a strong stance on national security and foreign policy, including military interventions in Ukraine and Syria. He has also been criticized for his crackdown on political dissent and human rights abuses. Putin remains a polarizing figure, admired by some for his strong leadership and criticized by others for his authoritarian tendencies.

"""

In [121]:
delimiter = "####"

prompt_template = f"""
Analyze this 3 articles, answer questions in one line.Take help from this {quiz_bank}

"""

In [122]:
from langchain.prompts import ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])

In [123]:
llm = AzureChatOpenAI(deployment_name='tnhgpt3' , verbose=False , temperature=0)

In [124]:
# parser
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
output_parser

StrOutputParser()

In [125]:
chain = chat_prompt | llm | output_parser


In [126]:
# taking all components and making reusable as one piece
def assistant_chain(
    system_message,
    human_template="{question}",
    llm=llm,
    output_parser=StrOutputParser()):
  
  chat_prompt = ChatPromptTemplate.from_messages([
      ("system", system_message),
      ("human", human_template),
  ])
  return chat_prompt | llm | output_parser

In [127]:
def eval_expected_words(
    system_message,
    question,
    expected_words,
    human_template="{question}",
    llm=llm,
    output_parser=StrOutputParser()):
    
  assistant = assistant_chain(
      system_message,
      human_template,
      llm,
      output_parser)
    
  
  answer = assistant.invoke({"question": question})
    
  print(answer)
    
  assert any(word in answer.lower() \
             for word in expected_words), \
    f"Expected the assistant questions to include \
    '{expected_words}', but it did not"

In [160]:
############## Test1 

question  = "what was putins foreign policy towards india"

expected_words = ["President","Russia", "Prime Minister", "national security",
 "foreign policy", "Ukraine", "Syria", "political dissent", "human rights abuses", "authoritarian"]
 
expected_words = [fruit.lower() for fruit in expected_words]

In [161]:
eval_expected_words(
    prompt_template,
    question,
    expected_words
)

The provided articles do not mention Vladimir Putin's specific foreign policy towards India.


In [162]:
############### Test 2 

question  = "what was putins foreign policy towards india"

expected_words = ["President", "United States", "Vice President", "Barack Obama", "Senator", "Delaware", 
"COVID-19 pandemic", "economy", "climate change", "racial equity", "diplomacy"]
 
expected_words = [fruit.lower() for fruit in expected_words]

In [163]:
eval_expected_words(
    prompt_template,
    question,
    expected_words
)

The provided articles do not mention Vladimir Putin's specific foreign policy towards India.


AssertionError: Expected the assistant questions to include     '['president', 'united states', 'vice president', 'barack obama', 'senator', 'delaware', 'covid-19 pandemic', 'economy', 'climate change', 'racial equity', 'diplomacy']', but it did not